In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
#! pip install torchaudio

In [5]:
import os
import csv
import time
import librosa

import warnings
import pandas as pd
from models import *
from utils import *
import torch
import torchaudio
import numpy as np
import torch.optim as optim

warnings.filterwarnings('ignore')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
%%time 

limit = 80
params = {'batch_size': 20,
          'shuffle': True,
          'num_workers': 10,
          'drop_last': True}


tokenizer = tokenizer_librispeech(limit=800)
training_set = LibriSpeechDataset(limit=limit, tokenizer=tokenizer, n_frames=600)
my_dataloader = data.DataLoader(training_set, **params)

steps_per_epoch = len(training_set) // params['batch_size']

vocab_tar_size = len(tokenizer.word_index)
embedding_dim_target = 100

 ===== They are 800 transcriptions in the dataset. ===== 
CPU times: user 481 ms, sys: 141 ms, total: 622 ms
Wall time: 907 ms


In [4]:
# 'bahdanau_basic'
# 'bahdanau_audio'
# 'luong_dot'
# 'luong_concat'
# 'luong_general'
# 'super_head'

In [20]:
encoder = EncoderCONV2DRNN(batch_size=params['batch_size'], device=device, hidden_size=64).to(device)
decoder = DecoderATTRNN1(vocab_tar_size, embedding_dim_target, dec_units=64, batch_sz=params['batch_size'], 
                         hidden_size=64, method='bahdanau_audio').to(device)

encoder_optimizer = optim.Adam(encoder.parameters())
decoder_optimizer = optim.Adam(decoder.parameters())

criterion = nn.NLLLoss()

In [18]:
A = torch.randn(20, 198, 64)
B = torch.randn(20, 198, 64)

#A + B

In [22]:
# Train the model
global_trainer(1, my_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer,
                                    criterion, device, params['batch_size'], tokenizer )

 ====== ====== ====== ====== ====== ======
      The model has 2006039 parameters
 ====== ====== ====== ====== ====== ======


Epoch 1:   0%|                    | 0/4 [00:02<?, ?it/s]


RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.

In [ ]:
mfccs, references = training_set[1]
references = [tokenizer.index_word[ind] for ind in references.numpy()]
evaluate(mfccs.unsqueeze(0), references, 40, encoder, decoder, targ_lang=tokenizer, 
          device=device, beam_search=True)

In [ ]:
A = torch.randn(20, 198, 1)
m = torch.nn.Conv1d(198, 24, 5, padding=2)

In [ ]:
m(A).shape